# [실습3] Langchain으로 문서 기반 챗봇 만들기 - PDF

## 실습 목표
---
[실습3] RAG를 위한 Vector Score, Retriever 에서 학습한 내용을 바탕으로 LangChain을 활용해서 입력된 문서를 요약해서 Context로 활용하는 챗봇을 개발합니다.

## 실습 목차
---

1. **교통 3대 혁신 전략 문서 벡터화:** RAG 챗봇에서 활용하기 위해 교통 3대 혁신 전략 파일을 읽어서 벡터화하는 과정을 실습합니다.

2. **RAG 체인 구성:** 이전 실습에서 구성한 미니 RAG 체인을 응용해서 간단한 교통 3대 혁신 전략 문서 기반 RAG 체인을 구성합니다.

3. **챗봇 구현 및 사용:** 구성한 RAG 체인을 활용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현하고 사용해봅니다.

## 실습 개요
---
RAG 체인을 활용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

- OpenAI를 통해 gpt-4o-mini 모델을 불러옵니다.

## 1. 문서 벡터화
- RAG 챗봇에서 활용하기 위해 교통 3대 혁신 전략 파일을 읽어서 벡터화하는 과정을 실습합니다.

먼저, gpt-4o-mini 모델을 사용하는 ChatOpenAI 객체와 OpenAIEmbeddings 객체를 생성합니다.

In [2]:
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

다음으로, PDF 문서를 불러와서 벡터화 해보겠습니다.
- 출처: 국토교통부의 교통 분야 3대 혁신 전략 보도자료
  - https://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?lcmspage=1&id=95089349
- 이 보고서는 2024년 교통 분야에서 3대 혁신 추진에 대해 다루는 보고서입니다. 
- 이를 활용해서 해당 정보를 빠르게 파악할 수 있도록 돕는 챗봇을 만들어야 하는 상황이라고 가정해 봅시다.

먼저, LangChain의 `PyPDFLoader`를 활용해서 교통 3대 혁신 전략 보고서의 텍스트를 추출하고, 페이지 별로 `Document`를 생성하여 저장합니다.

In [6]:
doc_path = "../data/교통_3대_혁신_전략.pdf"
loader = PyPDFLoader(doc_path)
docs = loader.load()

In [ ]:
docs

생성된 Document의 수를 확인해봅시다.

In [ ]:
print(len(docs))

다음으로, 각 Document의 길이를 확인해봅시다.

In [ ]:
doc_len = [len(doc.page_content) for doc in docs]
print(doc_len)

1천자 미만의 문서가 대부분인걸 확인할 수 있습니다. 하지만 실제로는 이대로 그냥 사용할 경우, Context를 조절하는 것도 중요한 RAG의 요소입니다.
우선은 이대로 RAG 체인을 구성해 봅시다.

## 2. RAG 체인 구성
RAG 체인을 구성하기 위해 `Document`를 `OpenAIEmbeddings`를 활용해 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.
- 변환 및 저장 과정은 약 3분 정도 소요됩니다.

In [9]:
vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings
)

In [10]:
db_retriever = vectorstore.as_retriever()

이전 실습에서 구성한 미니 RAG Chain과 비슷하게 Chain을 구성해 봅시다.
- 이번 챗봇의 역할은 교통 정보 제공을 위한 챗봇으로 고정했으므로, 역할을 별도로 인자로 전달할 필요가 없습니다.
- `RunnablePassthrough()`는 Chain의 이전 구성 요소에서 전달된 값을 그대로 전달하는 역할을 수행합니다.

### [TODO] pipe를 통해 체인을 구성해보세요.

In [11]:
def get_retrieved_text(docs):
    result = "\n".join([doc.page_content for doc in docs])
    return result

def init_chain():
    messages_with_contexts = [
        ("system", "당신은 교통 정보를 제공하기 위한 친절한 지원 챗봇입니다. 사용자가 입력하는 정보를 바탕으로 질문에 답하세요."),
        ("human", "정보: {context}.\n{question}."),
    ]

    prompt_with_context = ChatPromptTemplate.from_messages(messages_with_contexts)

    # 체인 구성
    # context에는 질문과 가장 비슷한 문서를 반환하는 db_retriever에 get_retrieved_text를 적용한 chain의 결과값이 전달됩니다.
    qa_chain = (
        {"context": db_retriever | get_retrieved_text, "question": RunnablePassthrough()}
        | ______________
        | ______________
        | ______________
    )
    
    return qa_chain

In [12]:
qa_chain = init_chain()

Chain 구성이 완료되었습니다.

## 3. 챗봇 구현 및 사용
- 구성한 RAG 체인을 활용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현하고 사용해봅니다.

방금 구현한 RAG Chain을 사용해서 교통 3대 혁신 전략 문서 기반 챗봇을 구현해볼 것입니다. 

그 전에, 별도로 RAG 기능을 추가하지 않은 LLM과 답변의 퀄리티를 비교해 봅시다.

### [TODO] 체인의 구성요소를 채워보세요.

In [13]:
messages_with_variables = [
    ("system", "당신은 교통 정보를 제공하기 위한 친절한 지원 챗봇입니다."),
    ("human", "{question}."),
]
prompt = ____________________________
parser = ____________________________
chain = prompt | llm | parser

In [ ]:
print(chain.invoke("지하 고속도로 사업 정보에 대해 알려줘"))

In [ ]:
print(qa_chain.invoke("지하 고속도로 사업 정보에 대해 알려줘. 반드시 한글로 대답해"))

일반 체인은 아무런 출처가 없는 답변을 생성한 반면, RAG 기능을 추가한 챗봇은 데이터를 기반으로 상대적으로 정확한 답변을 하는 것을 확인할 수 있습니다. 

이제 챗봇을 한번 사용해 봅시다.

In [ ]:
qa_chain = init_chain()
while True:
    question = input("질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "종료":
        break
    else:
        result = qa_chain.invoke(question)
        print(result)

저희는 이전 챕터에서 구현한 챗봇이 가지고 있는 문제점 중 '문서나 데이터 기반 추론이 불가능하다.'를 완화했습니다.